## Plot train/test error across different models/n train frames

In [1]:
import hydra
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import torch
from omegaconf import DictConfig

from lightning_pose.utils.io import return_absolute_data_paths
# from lightning_pose.utils.pca import KeypointPCA, compute_pca_reprojection_error
# from lightning_pose.utils.predictions import predict_dataset
from lightning_pose.utils.scripts import get_imgaug_transform, get_dataset, get_data_module, get_loss_factories
from lightning_pose.losses.losses import PCALoss

import sys
sys.path.append('/home/jovyan/tracking-diagnostics')
from diagnostics.handler import ModelHandler

In [2]:
def get_base_config(config_dir: str, config_name: str) -> DictConfig:
    assert(os.path.isdir(config_dir))
    hydra.initialize_config_dir(base_config_dir)
    cfg = hydra.compose(config_name="config")
    return cfg
    

In [3]:
from typing import List
def get_keypoint_names(csv_data: pd.DataFrame, header_rows: List[int]) -> List[str]:
    if header_rows == [0,1,2]:
        keypoint_names = [c[1] for c in csv_data.columns[1::2]]
    elif header_rows == [1,2]:
        keypoint_names = [c[0] for c in csv_data.columns[1::2]]
    return keypoint_names
# test for rick
# keypoint_names = get_keypoint_names(csv_data, cfg.data.header_rows)
# assert(len(get_keypoint_names(csv_data, cfg.data.header_rows))==17)

### define configuration

In [4]:
dataset_name = "rick-configs-1"
# dataset_name = "ibl-pupil-2"
# dataset_name = "ibl-paw-2"
base_config_dir = "/home/jovyan/rick-configs-1"
base_save_dir = "/home/jovyan/"

# hydra.initialize_config_dir(base_config_dir)
# cfg = hydra.compose(config_name="config")
cfg = get_base_config(base_config_dir, "config")

# load ground truth labels
csv_file = os.path.join(cfg.data.data_dir, cfg.data.csv_file)
csv_data = pd.read_csv(csv_file, header=list(cfg.data.header_rows))
keypoints_gt = csv_data.iloc[:, 1:].to_numpy().reshape(csv_data.shape[0], -1, 2)

keypoint_names = get_keypoint_names(csv_data, cfg.data.header_rows)

# if cfg.data.header_rows == [0,1,2]:
#     keypoint_names = [c[1] for c in csv_data.columns[1::2]]
# elif cfg.data.header_rows == [1,2]:
#     keypoint_names = [c[0] for c in csv_data.columns[1::2]]

In [13]:
new_kp_names = [kp for kp in keypoint_names if kp not in ['obs_top','obsHigh_bot','obsLow_bot']]
print(new_kp_names)

['paw1LH_top', 'paw2LF_top', 'paw3RF_top', 'paw4RH_top', 'tailBase_top', 'tailMid_top', 'nose_top', 'paw1LH_bot', 'paw2LF_bot', 'paw3RF_bot', 'paw4RH_bot', 'tailBase_bot', 'tailMid_bot', 'nose_bot']


In [11]:
keypoint_names

['paw1LH_top',
 'paw2LF_top',
 'paw3RF_top',
 'paw4RH_top',
 'tailBase_top',
 'tailMid_top',
 'nose_top',
 'obs_top',
 'paw1LH_bot',
 'paw2LF_bot',
 'paw3RF_bot',
 'paw4RH_bot',
 'tailBase_bot',
 'tailMid_bot',
 'nose_bot',
 'obsHigh_bot',
 'obsLow_bot']

In [6]:
cfg

{'training': {'train_batch_size': 16, 'val_batch_size': 16, 'test_batch_size': 16, 'train_prob': 0.8, 'val_prob': 0.1, 'train_frames': 1, 'num_gpus': 0, 'num_workers': 8, 'early_stop_patience': 3, 'unfreezing_epoch': 25, 'dropout_rate': 0.1, 'min_epochs': 125, 'max_epochs': 1000, 'log_every_n_steps': 1, 'check_val_every_n_epoch': 10, 'gpu_id': 0, 'unlabeled_sequence_length': 16, 'rng_seed_data_pt': 42, 'rng_seed_data_dali': 42, 'rng_seed_model_pt': 42, 'limit_train_batches': 10, 'multiple_trainloader_mode': 'max_size_cycle', 'profiler': 'simple', 'accumulate_grad_batches': 2, 'lr_scheduler': 'multisteplr', 'lr_scheduler_params': {'multisteplr': {'milestones': [100, 200, 300], 'gamma': 0.5}}}, 'losses': {'pca_multiview': {'log_weight': 0.0, 'components_to_keep': 3, 'empirical_epsilon_percentile': 1.0, 'empirical_epsilon_multiplier': 1.0, 'epsilon': None, 'error_metric': 'reprojection_error'}, 'pca_singleview': {'log_weight': 0.0, 'components_to_keep': 0.99, 'empirical_epsilon_percentile

What do we want to search over? be more faithful to the actual hydra structure so it's easy to search
have model groups. each group may have completely different parameters. including modelname etc. so can have a dict of dicts. 
the quantities to compute are a separate issue. we just need to get the model handlers.
each model is copying the config and overwriting it with command line args. so each one will be like {["data"]: 6} or something like that.

In [7]:
# save_dir = os.path.join(base_save_dir, dataset_name)
save_dir = "/home/jovyan/lightning-pose"

loss_type = "pca_singleview" # Change for different loss functions
error_metric = "reprojection_error" # for pca only
# define models
to_compute = "pca_singleview" # rmse | pca_multiview | pca_singleview | unimodal_mse
model_name = "pca-singleview-heatmap"
train_frames = 75 # 000
#pca_singleview_epsilon = 15.
#pca_multiview_epsilon = None
empirical_epsilon_multiplier = 1.0 # can hack this if want to sweep over multipliers/losses
model_type = "heatmap"
#rng_seed = 42

loss_weight_dict = {
    'supervised': [None],#[11.5],
    #'temporal': [None],
    'pca_singleview': [0.25,0.5,0.75,1.0,1.25,1.5],# was [6.5,6.75,7.0,7.25,7.5,8.0,11.5], # Hacking. this is epsilon
    #'pca_multiview': [1.0,1.25,1.5,2.0,4.0]# <-hack for now, [7.0,7.5,10.5] #<- actual log weights. #[1.0,1.5,2.0,3.0] <-hack for now, it's not weight, it's empirical epsilon multiplier, #[2.0, 1.0, 0.0, -1.0],

}
# loss_weight_dict['pca_singleview'].sort(reverse=True)

pca_obj = None
datamodule = None
model_cfg = cfg.copy()
model_cfg.training.train_frames = train_frames
model_cfg.model.losses_to_use = [loss_type]
if to_compute == 'rmse':
    y_label = 'RMSE per bodypart'
elif to_compute == 'pca_multiview' or to_compute == 'pca_singleview':
    y_label = 'PCA reprojection error'
    from lightning_pose.utils.pca import KeypointPCA
    data_dir, video_dir = return_absolute_data_paths(data_cfg=model_cfg.data)
    imgaug_transform = get_imgaug_transform(cfg=model_cfg)
    dataset = get_dataset(cfg=model_cfg, data_dir=data_dir, imgaug_transform=imgaug_transform)
    data_module = get_data_module(cfg=model_cfg, dataset=dataset, video_dir=video_dir)
    data_module.setup()
    # compute pca params
    loss_factories = get_loss_factories(cfg=model_cfg, data_module=data_module)
    pca_loss = loss_factories["unsupervised"].loss_instance_dict[loss_type]
    #pca_loss = PCALoss(loss_name=loss_type, loss_weight=1.0, loss_weight_dict=loss_weight_dict, empirical_epsilon_multiplier=empirical_epsilon_multiplier)
    #pca_obj = KeypointPCA(loss_type=loss_type, error_metric=error_metric, data_module=data_module, mirrored_column_matches=model_cfg.data.mirrored_column_matches, columns_for_singleview_pca=model_cfg.data.columns_for_singleview_pca)
    #pca_obj()
elif to_compute == 'unimodal_mse':
    y_label = 'Unimodal MSE'
elif to_compute == 'temporal_norm':
    raise NotImplementedError
    
# store results here
if to_compute == "pca_singleview":
    # remove obstacle keypoints
    keypoint_names = [kp for kp in keypoint_names if kp not in ['obs_top','obsHigh_bot','obsLow_bot']]

metrics_collected = {bp: [] for bp in keypoint_names}
cols_collected = []

# loop over models and compute metric of interest
for loss_type, loss_weights in loss_weight_dict.items():
    for loss_weight in loss_weights:

        # find model checkpoint
        model_cfg = cfg.copy()
        model_cfg.training.train_frames = train_frames
        # Dan -- edit the below with numbers
        model_cfg.training.rng_seed_data_pt = 42
        model_cfg.training.rng_seed_data_dali = 42
        model_cfg.training.rng_seed_model_pt = 42
        model_cfg.model.model_name = model_name
        model_cfg.model.model_type = model_type

        # put model-specific config info here
        if loss_type == 'supervised': # edits above are because of our use of 10.5 weight for the "supervised" loss
            model_cfg.model.model_name =  "supervised-heatmap-75" # "pca-singleview-heatmap" # was "supervised_50"
            model_cfg.model.losses_to_use = [] # ["pca_singleview"] # was None
            # hacks below
            # model_cfg.losses["pca_singleview"].log_weight = loss_weight #  <- TODO: just hack for now, should be loss_weight
             #model_cfg.losses["pca_singleview"].empirical_epsilon_multiplier = empirical_epsilon_multiplier # TODO: warning, it's a hack. should be empirical_epsilon_multiplier

        else:
            model_cfg.model.losses_to_use = [loss_type]
            model_cfg.losses[loss_type].log_weight = 7.25 #  <- TODO: just hack for now, should be loss_weight
            model_cfg.losses[loss_type].epsilon = loss_weight # TODO: hack; was pca_multiview_epsilon
            model_cfg.losses[loss_type].empirical_epsilon_multiplier = empirical_epsilon_multiplier # TODO: warning, it's a hack. should be empirical_epsilon_multiplier
#                 print(model_cfg.losses)
            if loss_type in ["pca_multiview","pca_singleview"]:
                model_cfg.losses[loss_type].error_metric = error_metric

        try:
            print(save_dir)
            handler = ModelHandler(save_dir, model_cfg, verbose=True)
        except FileNotFoundError:
            print('did not find %s model for train_frames=%i' % (loss_type, train_frames))
            continue

        try:
            result = handler.compute_metric(
                to_compute, 'predictions.csv',
                keypoints_true=keypoints_gt, pca_loss_obj=pca_loss, datamodule=datamodule)
        except FileNotFoundError:
            print('could not find model predictions')
            continue

        if loss_type == 'supervised':
            for loss_type_, loss_weights_ in loss_weight_dict.items():
                if loss_type_ == 'supervised':
                    # make a supervised entry, but not under this name
                    continue
                else:
                    cols_collected.append('%s_s' % loss_type_)
                    for b, bodypart in enumerate(keypoint_names):
                        metrics_collected[bodypart].append(result[:, b])
        else:
            #if len(loss_weight.split('.')[-1]) == 1:
            cols_collected.append('%s_%s' % (loss_type, loss_weight))
            #cols_collected.append('%s_%.1f' % (loss_type, loss_weight))
            for b, bodypart in enumerate(keypoint_names):
                metrics_collected[bodypart].append(result[:, b])

# collect results                
results_df = []
for bodypart in keypoint_names:
    dict_tmp = {
        'bodypart': bodypart,
        #'rng_seed': rng_seed,
        'eval_mode': handler.pred_df.iloc[:, -1].to_numpy(),
        'img_file': handler.pred_df.iloc[:, 0], # TODO: fix, this is wrong. should be a str not a float
    }
    for col_name, metric in zip(cols_collected, metrics_collected[bodypart]):
        dict_tmp[col_name] = metric
    results_df.append(pd.DataFrame(dict_tmp))

results_df = pd.concat(results_df)

Number of labeled images in the full dataset (train+val+test): 1045
Size of -- train set: 75, val set: 104, test set: 105


[/opt/dali/dali/operators/reader/loader/video_loader.h:177] ``file_list_include_preceding_frame`` is set to False (or not set at all). In future releases, the default behavior would be changed to True.
[/opt/dali/dali/operators/reader/nvdecoder/nvdecoder.cc:82] Warning: Decoding on a default stream. Performance may be affected.


Number of labeled images in the full dataset (train+val+test): 1045
Size of -- train set: 75, val set: 104, test set: 105
Results of running PCA (pca_singleview) on keypoints:
Kept 9/14 components, and found:
Explained variance ratio: [0.473 0.213 0.111 0.1   0.036 0.027 0.018 0.008 0.007 0.005 0.    0.
 0.    0.   ]
Variance explained by 9 components: 0.994
/home/jovyan/lightning-pose


/home/jovyan/lightning-pose/lightning_pose/losses/losses.py:295: UserWarning: Using empirical epsilon=0.067 * multiplier=1.000 -> total=0.067 for pca_singleview loss
  warnings.warn(


/home/jovyan/lightning-pose/multirun/2022-03-26/22-44-51/2
unmatched keys: [current vs requested]
model_name: pca-singleview-heatmap vs supervised-heatmap-75
losses_to_use: ['pca_singleview'] vs []

/home/jovyan/lightning-pose/multirun/2022-03-26/22-44-51/4
unmatched keys: [current vs requested]
model_name: pca-singleview-heatmap vs supervised-heatmap-75
losses_to_use: ['pca_singleview'] vs []

/home/jovyan/lightning-pose/multirun/2022-03-26/22-44-51/0
unmatched keys: [current vs requested]
model_name: pca-singleview-heatmap vs supervised-heatmap-75
losses_to_use: ['pca_singleview'] vs []

/home/jovyan/lightning-pose/multirun/2022-03-26/22-44-51/1
unmatched keys: [current vs requested]
model_name: pca-singleview-heatmap vs supervised-heatmap-75
losses_to_use: ['pca_singleview'] vs []

/home/jovyan/lightning-pose/multirun/2022-03-26/22-44-51/3
unmatched keys: [current vs requested]
model_name: pca-singleview-heatmap vs supervised-heatmap-75
losses_to_use: ['pca_singleview'] vs []

/home

IndexError: index 7 is out of bounds for axis 1 with size 7

In [ ]:
cols_collected

In [ ]:
if to_compute == "pca_multiview":
    # assert nans where we think they should be
    assert(~np.any(np.isnan(result[:, np.arange(7)])))
    assert(~np.any(np.isnan(result[:, np.arange(8,15)])))
    assert(np.all(np.isnan(result[:, np.arange(15, 34)])))

In [ ]:
results_df.head()

### scatterplots for a pair of models

In [ ]:
sns.set_context('talk')
sns.set_style('whitegrid')

eval_mode = 'train'

fig = px.scatter(
    results_df[results_df.eval_mode==eval_mode], 
    # x='unimodal_mse_0', y='unimodal_mse_5.0',
    x='pca_multiview_7.0', y='pca_multiview_10.5',
    facet_col='bodypart',
    facet_col_wrap=3,
    hover_data=['img_file'],
    log_x=True,
    log_y=True,
    opacity=0.5,
#     trendline="ols",
    title='%s on %s data' % (y_label, eval_mode),
)
# draw a diagonal line
fig.update_traces(marker={'size': 5})

# if dataset_name == 'ibl-paw-2':
#     if to_compute == 'unimodal_mse':
#         mn, mx = 0.0001, 0.01
#     else:
#         mn, mx = 0.1, 80
# elif dataset_name == 'ibl-pupil-2':
#     if to_compute == 'unimodal_mse':
#         mn, mx = 0.0004, 0.005
#     else:
#         mn, mx = 0.1, 10
# elif dataset_name == 'ibl-fingers':
#     if to_compute == 'rmse':
#         mn, mx = 2, 40
#     if to_compute == 'pca_reproj':
#         mn, mx = 2, 40
# trace = go.Scatter(x=[mn, mx], y=[mn, mx], line_color="black", mode="lines")
# trace.update(legendgroup="trendline", showlegend=False)
# fig.add_trace(trace, row="all", col="all", exclude_empty_subplots=True)
fig.update_layout(width=800, height=1200)

fig.show()

### barplots across all models

In [ ]:
df_tmp = pd.melt(
    results_df, 
    id_vars=['bodypart', 'img_file', 'eval_mode'], 
    value_vars=cols_collected
)
def add_loss_name_col(row):
    return '_'.join(row['variable'].split('_')[:-1])
def add_loss_val_col(row):
    return row['variable'].split('_')[-1]
df_tmp['loss'] = df_tmp.apply(add_loss_name_col, axis=1)
df_tmp['loss_weight'] = df_tmp.apply(add_loss_val_col, axis=1)

In [ ]:
sns.set(context='talk', style='whitegrid', font_scale=1)

eval_mode = 'validation'

num_losses_to_plot = len(cols_collected) # can exclude vals here
# average over keypoints and frames
df_tmp_ = df_tmp[df_tmp.eval_mode==eval_mode]
g = sns.catplot(
    x='loss_weight', y='value', 
    #log=False,
    order=['s'] + [str(w) for w in loss_weight_dict['pca_singleview'][:num_losses_to_plot]],
    #kind='strip', dodge=True,
    kind='bar',
    col='loss',
    col_wrap=np.min([len(df_tmp_.loss.unique()), 3]), 
    data=df_tmp_,
    col_order=['temporal', 'pca_singleview'],
#     col='bodypart',
#     sharey=False,
#     data=df_tmp_[df_tmp_.loss=='pca_singleview'],
)
g.set_axis_labels('Loss weight', y_label)
#g.set_xticklabels(rotation=45, ha='center')

for ax in g.axes.flatten():
    ax.tick_params(axis='y', which='both', direction='out', length=4, left=True)
    ax.grid(b=True, which='both', color='gray', linewidth=0.1)

# if dataset_name == 'ibl-paw-2':
#     if to_compute == 'pca_reproj':
#         if eval_mode == 'unused':
#             g.set(ylim=[3.8, 4.8])
#         elif eval_mode == 'validation':
#             g.set(ylim=[4, 5.7])
#         elif eval_mode == 'train':
#             g.set(ylim=[4, 5.5])
# elif dataset_name == 'ibl-pupil-2':
#     if to_compute == 'rmse':
#         if eval_mode == 'validation':
#             g.set(ylim=[1.5, 2.4])
# #     if to_compute == 'unimodal_mse':
# #         g.set(ylim=[0.008, .015])
# elif dataset_name == 'ibl-fingers':
#     if to_compute == 'rmse':
#         if eval_mode == 'train':
#             g.set(ylim=[7, 14])
#         elif eval_mode == 'validation':
#             g.set(ylim=[10, 20])
#         elif eval_mode == 'test':
#             g.set(ylim=[14, 20])
#     elif to_compute == 'pca_reproj':
#         if eval_mode == 'test':
#             g.set(ylim=[12, 14.4])
            
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('%s on %s data' % (y_label, eval_mode))
plt.tight_layout()
plt.show()

In [ ]:
df_tmp_

In [ ]:
sns.set(context='talk', style='whitegrid', font_scale=1)

eval_mode = 'test'

# average over keypoints and frames
# filter weird bps
df_tmp = df_tmp[~df_tmp["bodypart"].isin(["obsLow_bot", "obsHigh_bot", "obs_top"])]
# select eval mode
df_tmp_ = df_tmp[df_tmp.eval_mode==eval_mode]
g = sns.boxenplot(
    y='loss_weight', x='value', # switched these around for horizontal boxenplot
    order=['s'] + [str(w) for w in loss_weight_dict['pca_singleview']], # was loss_weight_dict['pca_singleview']
#     kind='strip', dodge=True,
    data=df_tmp_, 
    orient="h",
)
print("bodyparts included: {}".format(pd.unique(df_tmp_.bodypart)))